# Advanced databases

## Stored procedures, user menagment, view
### dr  inż. Waldemar Bauer

## Stored Procedures (function)

- The procedure declared in the database engine.
- Used to perform repetitive tasks
- Uses different procedure language depending on the database engine, in PostgreSQL PL/pgSQL.


## Advantages and disadvantages  of using PostgreSQL stored procedures

**Advantages:**
- Reduces traffic between the application and the database server
- Increase user application performance
- Reusable 

**Disadvantages**
- Difficult to manage
- Hard to portable to other database management systems
- Heavy load on the database server

## Variables in stored procedures
```sql
variable_name data_type [:= expression];
```
Data type exampel: 
- INTEGER, 
- NUMERIC, 
- VARCHAR, 
- CHAR,
- TIME,
- DATE.

## Constant variable

``` sql
   pi CONSTANT NUMERIC := 3.14;
```

```sql
   start_time CONSTANT time := now();
```

## Block Structure
```sql
[ <<label>> ]  
[ DECLARE  
    declarations ]
BEGIN  
    statements;  
	...
END [ label ];  
```

## Exampel of use

```sql
DO $$
<<exampel_block>>
DECLARE
  exampel_value integer := 0;
  exampel_value2 numeric := 0.1;
BEGIN 
  
    RAISE NOTICE 'The current exampel value is %. And example value 2 is %', exampel_value, exampel_value2;

END exampel_block $$;
```
**Result:**
```
 The current exampel value is 0. And example value 2 is 0.1
```

## Subblock
```sql
DO $$ 
<<super_block>>
DECLARE
  exampel_value integer := 0;
BEGIN 
   exampel_value := exampel_value + 1;
   RAISE NOTICE 'The current exampel value is %', exampel_value;
   
   <<subblock1>>
   DECLARE 
       exampel_value integer := 0;
   BEGIN 
       exampel_value := exampel_value + 10;
       RAISE NOTICE 'The current exampel value in the subblock1 is %', exampel_value;
       RAISE NOTICE 'The current exampel value in the super block is %', super_block.exampel_value;
   END subblock1;
   
   <<subblock2>>
   DECLARE 
       exampel_value integer := 0;
   BEGIN 
       exampel_value := exampel_value + 100;
       RAISE NOTICE 'The current  exampel value in the subblock1 is %', exampel_value;
       --RAISE NOTICE 'The current  exampel value in the subblock2 is %', subblock1.exampel_value; --error
       RAISE NOTICE 'The current  exampel value in the super block is %', super_block.exampel_value;
   END subblock2;

   RAISE NOTICE 'The current value of counter in the outer block is %', exampel_value;
   
END super_block $$;
```

## Subblock results
```
The current exampel value is 1
The current exampel value in the subblock1 is 10
The current exampel value in the super block is 1
The current  exampel value in the subblock1 is 100
The current  exampel value in the super block is 1
The current value of counter in the outer block is 1
```

## Dubugging with Raise

```sql
RAISE level format;
```
level:

- DEBUG
- LOG
- NOTICE
- INFO
- WARNING
- EXCEPTION


## RAISE message use

```sql
DO $$ 
BEGIN 
  RAISE INFO 'information message %', now() ;
  RAISE LOG 'log message %', now();
  RAISE DEBUG 'debug message %', now();
  RAISE WARNING 'warning message %', now();
  RAISE NOTICE 'notice message %', now();
END $$;
```
```sql
set client_min_messages to 'debug'
```

## EXCEPTION  and ASSERT

**EXCEPTION**
```sql
DO $$ 
DECLARE
 value INTEGER := -1;
BEGIN 
  -- do something and check value
  -- ...
  -- report 
  RAISE EXCEPTION 'value less than 0: %', value 
		USING HINT = 'Check the value';
END $$;
```

**ASSERT**
```sql
DO $$ 
DECLARE
 value INTEGER := -1;
BEGIN 
  -- do something 
  -- ...
  -- report 
  Assert value > 0, 'value less than 0';
END $$;
```



##  CREATE \[OR REPLACE\] FUNCTION

```sql
CREATE [OR REPLACE]  FUNCTION function_name([p1 type,])
 [RETURNS type] AS
BEGIN
 -- do something
END;
LANGUAGE language_name;
```
All created functions are in the database memory and the user has access to them through the command:

```sql
select function_name([p1,])
```


##  Function example

```sql
CREATE OR REPLACE FUNCTION  add(val1 integer, val2 integer) RETURNS integer AS $$
BEGIN
RETURN val1 + val2;
END; $$
LANGUAGE PLPGSQL;
```
Exampel of use:
```sql 
select add(1,2) --3
select add(Cast(1.0 as integer), 3) -- 4
select add(0.2, 3) -- error
```

##  Function example alternative declaration

```sql
CREATE OR REPLACE FUNCTION add(val1 integer, val2 integer) RETURNS integer 
LANGUAGE PLPGSQL

AS $BODY$
BEGIN
RETURN val1 + val2;
END; $BODY$;
```

## Function overloade
```sql
CREATE OR REPLACE FUNCTION  add(val1 numeric, val2 numeric) RETURNS numeric AS $$
BEGIN
RETURN val1 + val2;
END; $$
LANGUAGE PLPGSQL;
```
Exampel of use:
```sql 
select add(1,2) -- 3
select add(0.2, 3.2) -- 3.4
select add(1.0 , 3) -- ??
```

## Function Parameters modifiers

Parameters modifiers:
- in
- out
- inout
- default

```sql
CREATE OR REPLACE  FUNCTION upplow_case(in word  VARCHAR, 
                                          OUT lowcase_word  VARCHAR , 
                                          OUT uppercase_word  VARCHAR) AS $$
BEGIN
uppercase_word := upper(word); 
lowcase_word := lower(word) ;
END; $$
LANGUAGE PLPGSQL;
```

Exampel of use:
```
select  upplow_case('Advance Database') -- return "("advance database","ADVANCE DATABASE")"
select * from upplow_case('Advance Database') -- "advance database", "ADVANCE DATABASE"
```


## Default modifiers

```sql
CREATE OR REPLACE  FUNCTION function_name(...,[in|inout] p1  type DEFAULT value,...) ...
```

```sql
CREATE OR REPLACE  FUNCTION upplow_case(in word  VARCHAR DEFAULT 'Test for Users', 
                                          OUT lowcase_word  VARCHAR , 
                                          OUT uppercase_word  VARCHAR) AS $$
BEGIN
uppercase_word := upper(word); 
lowcase_word := lower(word) ;
END; $$
LANGUAGE PLPGSQL;
```
 
```sql
select upplow_case()
select * from upplow_case()
select * from upplow_case('I like this mechanisms')
```
 

## Returns A Table
 
```sql
CREATE OR REPLACE FUNCTION get_actor_film (actor_first_name VARCHAR, actor_last_name VARCHAR) 
	RETURNS TABLE (
		film_title VARCHAR,
		film_release_year INT
) 
AS $$
BEGIN
	RETURN QUERY SELECT
		title,
		cast( release_year as integer)
	FROM
		film inner join film_actor using (film_id) 
		inner join actor using(actor_id)
	WHERE
		actor.first_name ILIKE actor_first_name and actor.last_name ILIKE actor_last_name ;
END; $$ 

LANGUAGE 'plpgsql';
```

Example

```sql
select * from get_actor_film('Ed',	'Chase')
```

##  Control flow

- IF
- CASE 
- loops:
    - LOOP
    - WHILE
    - FOR 

## IF

```sql
  IF condition THEN
	-- do something
  
  [ELSEIF condition2 THEN
	-- do something 2
   ...]
  [ELSE
      -- do something
  ]
  END IF;
```

## IF Example
```sql
DO $$
DECLARE
   a integer := 5;
   b integer := 5;
BEGIN 
  IF a > b THEN 
     RAISE NOTICE 'a is greater than b';
  ELSIF a < b THEN
     RAISE NOTICE 'a is less than b';
  ELSE
     RAISE NOTICE 'a is equal to b';
  END IF;
END $$;
```

## CASE

```sql
CASE
    WHEN boolean-expression-1 THEN
      statements
  [ WHEN boolean-expression-2 THEN
      statements
    ... 
  ]
  ELSE
      statements
END CASE;
```

 ## Example
 
 ```sql
   CREATE OR REPLACE FUNCTION get_actor_status (p_actor_id INTEGER) 
	RETURNS VARCHAR (25) AS $$ 
DECLARE
    total_films NUMERIC ; 
    status VARCHAR (25) ;
BEGIN
    SELECT
	INTO total_films count (f.film_id)
     FROM
	film f inner join film_actor fa USING(film_id)
     WHERE
	fa.actor_id = p_actor_id ; 
  
   CASE
      WHEN total_films > 50 THEN
         status = 'more than 50' ;
      WHEN total_films >= 30 THEN
	 	 status = '30-50' ;
	  When total_films >= 10 THEN
	 	 status = '10-29' ;
      ELSE
         status = 'less than 10' ;
   END CASE ;

   RETURN status ;
END ; $$ 
LANGUAGE plpgsql;
 ```

## Loops
**LOOP**
```sql
<<label>>
LOOP
   Statements;
   EXIT [<<label>>] WHEN condition;
END LOOP;
```
**WHILE**
```sql
[ <<label>> ]
WHILE condition LOOP
   statements;
END LOOP;
```
**FOR**
```sql
[ <<label>> ]
FOR target IN query LOOP
    statements
END LOOP [ label ];
```


## LOOP Exampel

```sql
CREATE OR REPLACE FUNCTION factorial_loop (n INTEGER) 
	RETURNS INTEGER AS $$ 
DECLARE
   counter INTEGER := 1 ; 
   i INTEGER := 1 ; 
BEGIN
	IF (n < 0) THEN
		RAISE EXCEPTION 'Argument value less then 0'
			using hint = 'Argument must be bigger or equale 0';
	END IF; 
	IF (n < 1)  THEN
		RETURN 1 ;
	END IF; 

	LOOP 
		SELECT counter + 1, i * counter INTO counter,	i ;
		EXIT WHEN counter > n;
	END LOOP ; 
	
	RETURN i ;
END;$$
LANGUAGE PLPGSQL;
```

## WHILE Exampel
```sql
CREATE OR REPLACE FUNCTION factorial_while (n INTEGER) 
	RETURNS INTEGER AS $$ 
DECLARE
   counter INTEGER := 1 ; 
   i INTEGER := 1 ; 
BEGIN
	IF (n < 0) THEN
		RAISE EXCEPTION 'Argument value less then 0'
			using hint = 'Argument must be bigger or equale 0';
	END IF; 
	IF (n < 1)  THEN
		RETURN 1 ;
	END IF; 
	
	WHILE counter <= n LOOP 
		SELECT counter + 1, i * counter INTO counter,	i ;
	END LOOP ; 
	
	RETURN i ;
END;$$
LANGUAGE PLPGSQL;
```

## FOR Exampel
```sql
CREATE OR REPLACE FUNCTION factorial_for (n INTEGER) 
	RETURNS INTEGER AS $$ 
DECLARE
   i INTEGER := 1 ; 
BEGIN
	IF (n < 0) THEN
		RAISE EXCEPTION 'Argument value less then 0'
			using hint = 'Argument must be bigger or equale 0';
	END IF; 
	IF (n < 1)  THEN
		RETURN 1 ;
	END IF; 
	
    FOR counter IN 1..n LOOP
        i:= i * counter;
	END LOOP ; 
	
	RETURN i ;
END;$$
LANGUAGE PLPGSQL;
```

# FOR with select
```sql
CREATE OR REPLACE FUNCTION for_loop_through_query(
   n INTEGER DEFAULT 10
) 
RETURNS VOID AS $$
DECLARE
    rec RECORD;
BEGIN
    FOR rec IN SELECT title 
	       FROM film 
	       ORDER BY title
	       LIMIT n 
    LOOP 
	RAISE NOTICE '%', rec.title;
    END LOOP;
END;
$$ LANGUAGE plpgsql;
```

## For with param

```sql
[ <<label>> ]
FOR row IN EXECUTE string_expression [ USING query_param [, ... ] ] 
LOOP
    statements
END LOOP [ label ];
```

## For with param exampel

```sql
CREATE OR REPLACE FUNCTION for_loop_query_param(
   year1 INTEGER,  year2 INTEGER
) 
RETURNS VOID AS $$
DECLARE
    rec RECORD;
    query text;
BEGIN
		
	query := 'SELECT title, release_year FROM film ';

	query := query || 'where release_year between $1 and $2';

	FOR rec IN EXECUTE query USING year1, year2 
        LOOP
	   RAISE NOTICE '% - %', rec.release_year, rec.title;
	END LOOP;
   
END;
$$ LANGUAGE plpgsql;
```

## Stored procedure
- In the previous slids, you saw how define and use functions in PostgreSQL. 
- Functions cannot execute transactions. 
- Inside a function you cannot open a new transaction.

For make a procedure we use:

```sql
CREATE [OR REPLACE] PROCEDURE procedure_name(parameter_list)
LANGUAGE language_name
AS $$
    stored_procedure_body;
$$;
```

## Exampel
```sql
CREATE OR REPLACE PROCEDURE transfer(INT, INT, DEC)
LANGUAGE plpgsql     
AS $$
BEGIN
    UPDATE accounts 
    SET balance = balance - $3
    WHERE id = $1;

    UPDATE accounts 
    SET balance = balance + $3
    WHERE id = $2;

    COMMIT;
END;
$$;
```

## VIEW

- A view helps simplify the complexity of a query because you can query a view, which is based on a complex query, using a simple SELECT statement.
- Like a table, you can grant permission to users through a view that contains specific data that the users are authorized to see.
- A view provides a consistent layer even the columns of underlying table changes.

```sql
CREATE [OR REPLACE] VIEW view_name AS query;
ALTER VIEW customer_master RENAME TO customer_info;
DROP VIEW IF EXISTS customer_info;

```

## Update view
A PostgreSQL view is updatable when it meets the following conditions:

- The defining query of the view must have exactly one entry in the FROM clause, which can be a table or another updatable view.
- The defining query must not contain one of the following clauses at the top level: GROUP BY, HAVING, LIMIT, OFFSET, DISTINCT, WITH, UNION, INTERSECT, and EXCEPT.
- The selection list must not contain any window function , any set-returning function, or any aggregate function such as SUM, COUNT, AVG, MIN, and MAX.

## Create view

```sql
CREATE OR REPLACE VIEW actors_films 
AS
SELECT
		first_name,	
        last_name,
		title,
        release_year
    FROM
        film inner join film_actor using (film_id) 
        inner join actor using(actor_id)
```

##  Roles Management

- Manage database access permissions
- A role can be a user or a group
- A role that has login right is called user
- A role may be a member of other roles, which are known as groups

Usefulle query to chcek roles and users:
```sql
SELECT * FROM pg_roles;
```

## Roles PRIVILEGES
- SUPERUSER
- CREATEDB
- CREATEROLE
- INHERIT
- LOGIN
- Oposit: NOSUPERUSER, NOCREATEDB, NOCREATEROLE,NOINHERIT, NOLOGIN

## SQL managment of roles

**Creating roles:**
```sql
CREATE ROLE role_name [PRIVILEGES [PRIVILEGES]];
```

**Change role:**
```sql
ALTER ROLE role_name [PRIVILEGES [PRIVILEGES]];
```

**Remove roles:**
```sql
DROP ROLE IF EXISTS role_name;
```

## Switch between roles

Set role context:
```sql
SET ROLE role_name;
```

Stopr use role context:
```sql
RESET ROLE role_name;
```


## User

- Special type of role
- Have privileges to login in to database
- May belong to a group


## SQL managment of users

**Create users:**
```sql
CREATE ROLE your_login WITH PASSWORD your_password [VALID UNTIL date];
-- or
CREATE USER your_login WITH ENCRYPTED PASSWORD your_password valid until date;
```
**Add user to group:**
```sql
GRANT group_role to user_role;
```

**Remove user from group:**
```sql
REVOKE group_role FROM user_role;
```


## SQL managment of users part 2

**Edit users:**
```sql
ALTER ROLE role_name WITH attribute_options;
```
**Add table to user:**
```sql
GRANT [permission_type[, permission_type]] ON [table_name [, table_name]] TO role_name;
```
User permission type:
- SELECT
- INSERT
- UPDATE
- DELETE
- DEFAULT PRIVILEGES
- ALL PRIVILEGES


## SQL managment of users part 3

**Add database to user::**
```sql
GRANT [permission_type[, permission_type]] ON DATABASE database_name TO username;
```
**Add schema to user:**
```sql
GRANT USAGE ON SCHEMA schema_name TO username;
```
**Set provoleages to all table in schema**
```sql
GRANT [permission_type[, permission_type]]  ON ALL TABLES IN SCHEMA schema_name TO username;
```

## SQL managment of users part 3
**Remove privilege from user:**
```sql
REVOKE  [permission_type[, permission_type]] ON [table_name [, table_name]] FROM wbauer;
```

**Drop user:**
```sql
DROP user IF EXISTS role_name;
```